# Web Scraping in Python: Gathering NBA Player Information

Erick Lu

March 22, 2020

## Introduction

In this project, I use Python’s `urllib` and `re` modules to “scrape” ESPN for stats on all the players in the NBA. I then use `pandas` to format the data into a DataFrame, and calculate some interesting statistics.

Web scraping is a useful technique for extracting data from websites that don’t offer formatted, raw data for download. You can write scripts in Python to automate the process of obtaining information from these websites, so that you don’t have to spend hours flipping through each page and copy-pasting.

As an example, I will be scraping data from the rosters for each team in the NBA (as of currently, March 2020) for data such as age, height, weight, and salary. I will also loop through each individual player's stats page and extract career averages such as points per game, free throw percentages, and more.

We can use the data to answer questions such as:

* Do factors such as age, height, weight, etc. correlate with player performance? (i.e. does height matter?)
* How much do better players cost? Can we model the average price of hiring a player given his performance? If so, what is the cost per increase in points per game?
* What is the average salary paid by each team in the NBA, and which player earns the most on each team?

I've also exported the data to a nicely organized csv file, accessible in the GitHub repo for this project if you would like to analyze it yourself. You can also run the python script `scrape_nba_player_stats.py` to re-scrape ESPN yourself to obtain the latest data.

In the following sections, I will describe how to navigate through ESPN, read the page sources using `urllib`, extract information using `re`, and organize player statistics in a `pandas` DataFrame.

## Scrape roster information for each NBA team

We will first take a look at the website to figure out which web pages we need to scrape information from. The teams page at https://www.espn.com/nba/teams looks like the following:

![ESPN_teams_webpage.png](images/ESPN_teams_webpage.png)

This looks very promising. All the team names are listed on this page, which means that they can easily be extracted from the page source. Let’s take a look at the page source to see if we can find the URLs for each team's roster:

![ESPN_teams_source.png](images/ESPN_teams_source.png)

It looks like URLs for each of the teams rosters are listed in the page source with the following format: https://www.espn.com/nba/team/roster/_/name/team/team-name, as shown in the highlighted portion of the image above. Given that these all follow the same format, we can use regular expressions to pull out a list of all the team names from the page source, and then construct the roster URLs using the format above. Start by importing the `urllib` and `re` modules:

In [1]:
import re
import urllib
from time import sleep

Now, let’s create a function that will extract all the team names from http://www.espn.com/nba/teams and construct the each of the team's roster URLs:

In [14]:
# This method finds the urls for each of the rosters in the NBA using regexes.
def build_team_urls():
    # Open the espn teams webpage and extract the names of each roster available.
    f = urllib.request.urlopen('http://www.espn.com/nba/teams')
    teams_source = f.read().decode('utf-8')
    teams = dict(re.findall("www\.espn\.com/nba/team/_/name/(\w+)/(.+?)\",", teams_source))
    # Using the names of the rosters, create the urls of each roster
    roster_urls = []
    for key in teams.keys():
        # each roster webpage follows this general pattern.
        roster_urls.append('http://www.espn.com/nba/team/roster/_/name/' + key + '/' + teams[key])
        teams[key] = str(teams[key])
    return dict(zip(teams.values(), roster_urls))


In [28]:
rosters = build_team_urls()
rosters

{'atlanta-hawks': 'http://www.espn.com/nba/team/roster/_/name/atl/atlanta-hawks',
 'boston-celtics': 'http://www.espn.com/nba/team/roster/_/name/bos/boston-celtics',
 'brooklyn-nets': 'http://www.espn.com/nba/team/roster/_/name/bkn/brooklyn-nets',
 'charlotte-hornets': 'http://www.espn.com/nba/team/roster/_/name/cha/charlotte-hornets',
 'chicago-bulls': 'http://www.espn.com/nba/team/roster/_/name/chi/chicago-bulls',
 'cleveland-cavaliers': 'http://www.espn.com/nba/team/roster/_/name/cle/cleveland-cavaliers',
 'dallas-mavericks': 'http://www.espn.com/nba/team/roster/_/name/dal/dallas-mavericks',
 'denver-nuggets': 'http://www.espn.com/nba/team/roster/_/name/den/denver-nuggets',
 'detroit-pistons': 'http://www.espn.com/nba/team/roster/_/name/det/detroit-pistons',
 'golden-state-warriors': 'http://www.espn.com/nba/team/roster/_/name/gs/golden-state-warriors',
 'houston-rockets': 'http://www.espn.com/nba/team/roster/_/name/hou/houston-rockets',
 'indiana-pacers': 'http://www.espn.com/nba/t

The function `build_team_urls()` returns a dictionary that matches team names with their corresponding roster URL. Given this information, we can systematically loop through all of the team rosters and use regular expressions to extract player information from each roster.

In order to figure out how to scrape the rosters, let’s take a look at the Golden State Warriors' roster page:

![GSW_roster_webpage.png](images/GSW_roster_webpage.png)

Information for each player is nicely laid out in a table, meaning that the data is likely easily obtainable using regular expressions. Taking a look at the page source reveals that each player’s name and information are all provided in blocks of what apppear to be `json`, highlighted below:

![GSW_roster_source.png](images/GSW_roster_source.png)

Given the standardized format of the data for each player, this information is extractable using regular expressions. Let's walk through what the function does. First, we read in the roster webpage using `urllib.request.urlopen`:

In [6]:
url = "https://www.espn.com/nba/team/roster/_/name/gs/golden-state-warriors"
f = urllib.request.urlopen(url)
roster_source = f.read().decode('utf-8')

Then, we construct the regex that will return information for each of the players on the roster webpage.

In [8]:
player_regex = ('\{\"name\"\:\"(\w+\s\w+)\",\"href\"\:\"http\://www\.espn\.com/nba/player/.*?\",(.*?)\}')
player_regex
player_info = re.findall(player_regex, roster_source)
player_info[0:4]

[('Ky Bowman',
  '"uid":"s:40~l:46~a:4065635","guid":"d0ef63e951bb5f842b7357521697dc62","id":"4065635","height":"6\' 1\\"","weight":"187 lbs","age":22,"position":"PG","jersey":"12","salary":"$350,189","birthDate":"06/17/97","headshot":"https://a.espncdn.com/i/headshots/nba/players/full/4065635.png","lastName":"Ky Bowman","experience":0,"college":"Boston College"'),
 ('Marquese Chriss',
  '"uid":"s:40~l:46~a:3907487","guid":"a320ecf1d6481b7518ddc1dc576c27b4","id":"3907487","height":"6\' 9\\"","weight":"240 lbs","age":22,"position":"C","jersey":"32","salary":"$654,469","birthDate":"07/02/97","headshot":"https://a.espncdn.com/i/headshots/nba/players/full/3907487.png","lastName":"Marquese Chriss","experience":3,"college":"Washington","birthPlace":"Sacramento, CA"'),
 ('Stephen Curry',
  '"uid":"s:40~l:46~a:3975","guid":"5dda51f150c966e12026400b73f34fad","id":"3975","height":"6\' 3\\"","weight":"185 lbs","age":32,"position":"PG","jersey":"30","salary":"$40,231,758","birthDate":"03/14/88","h

As you can see, `player_info` is a list of tuples, in which each player name is paired with a set of information (height, weight, age, etc.) that is organized in `json` format. We can use the `json` module in Python to convert the information into a `dict`:

In [9]:
import json
draymond = json.loads("{"+player_info[3][1]+"}")
draymond

{'age': 30,
 'birthDate': '03/04/90',
 'birthPlace': 'Saginaw, MI',
 'college': 'Michigan State',
 'experience': 7,
 'guid': 'de360720e41625f28a6bb5ff82616cb1',
 'headshot': 'https://a.espncdn.com/i/headshots/nba/players/full/6589.png',
 'height': '6\' 6"',
 'id': '6589',
 'jersey': '23',
 'lastName': 'Draymond Green',
 'position': 'PF',
 'salary': '$18,539,130',
 'uid': 's:40~l:46~a:6589',
 'weight': '230 lbs'}

In the example above, all of the pertinent information for Draymond Green is now stored into a Python dictionary. Now that we know we can feasibly extract the player information, let's use the snippets of code above to construct a function in which we loop through each roster in `rosters` and scrape information for each player:

In [47]:
def get_player_info(roster_url):
    f = urllib.request.urlopen(roster_url)
    roster_source = f.read().decode('utf-8')
    sleep(0.5)
    player_regex = ('\{\"name\"\:\"(\w+\s\w+)\",\"href\"\:\"http\://www\.espn\.com/nba/player/.*?\",(.*?)\}')
    player_info = re.findall(player_regex, roster_source)
    player_dict = dict()
    for player in player_info:
        player_dict[player[0]] = json.loads("{"+player[1]+"}")
    return(player_dict)

We will now loop through each team in `rosters` and run `get_player_info()`, storing the output in a dictionary called `all_players`:

In [49]:
all_players = dict()
for team in rosters.keys():
    print("Gathering player info for team: " + team)
    all_players[team] = get_player_info(rosters[team])

Gathering player info for team: boston-celtics
Gathering player info for team: brooklyn-nets
Gathering player info for team: new-york-knicks
Gathering player info for team: philadelphia-76ers
Gathering player info for team: toronto-raptors
Gathering player info for team: chicago-bulls
Gathering player info for team: cleveland-cavaliers
Gathering player info for team: detroit-pistons
Gathering player info for team: indiana-pacers
Gathering player info for team: milwaukee-bucks
Gathering player info for team: atlanta-hawks
Gathering player info for team: charlotte-hornets
Gathering player info for team: miami-heat
Gathering player info for team: orlando-magic
Gathering player info for team: washington-wizards
Gathering player info for team: denver-nuggets
Gathering player info for team: minnesota-timberwolves
Gathering player info for team: oklahoma-city-thunder
Gathering player info for team: portland-trail-blazers
Gathering player info for team: utah-jazz
Gathering player info for team

The `all_players` dictionary should be a dictionary of dictionaries of dictionaries. This sounds complicated, but let's walk through what it looks like. The first level of keys should correspond to teams:

In [50]:
all_players.keys()

dict_keys(['boston-celtics', 'brooklyn-nets', 'new-york-knicks', 'philadelphia-76ers', 'toronto-raptors', 'chicago-bulls', 'cleveland-cavaliers', 'detroit-pistons', 'indiana-pacers', 'milwaukee-bucks', 'atlanta-hawks', 'charlotte-hornets', 'miami-heat', 'orlando-magic', 'washington-wizards', 'denver-nuggets', 'minnesota-timberwolves', 'oklahoma-city-thunder', 'portland-trail-blazers', 'utah-jazz', 'golden-state-warriors', 'la-clippers', 'los-angeles-lakers', 'phoenix-suns', 'sacramento-kings', 'dallas-mavericks', 'houston-rockets', 'memphis-grizzlies', 'new-orleans-pelicans', 'san-antonio-spurs'])

Within a team, the keys should correspond to player names. Let's zoom in on the LA Lakers:

In [51]:
all_players["los-angeles-lakers"].keys()

dict_keys(['Kostas Antetokounmpo', 'Avery Bradley', 'Devontae Cacok', 'Alex Caruso', 'Quinn Cook', 'Anthony Davis', 'Jared Dudley', 'Danny Green', 'Dwight Howard', 'LeBron James', 'Kyle Kuzma', 'JaVale McGee', 'Markieff Morris', 'Rajon Rondo', 'Dion Waiters'])

Now we can choose which player to look at. Let's choose LeBron James as an example:

In [53]:
all_players["los-angeles-lakers"]["LeBron James"]

{'age': 35,
 'birthDate': '12/30/84',
 'birthPlace': 'Akron, OH',
 'experience': 16,
 'guid': '1f6592b3ff53d3218dc56038d48c1786',
 'headshot': 'https://a.espncdn.com/i/headshots/nba/players/full/1966.png',
 'height': '6\' 9"',
 'id': '1966',
 'jersey': '23',
 'lastName': 'LeBron James',
 'position': 'SF',
 'salary': '$37,436,858',
 'uid': 's:40~l:46~a:1966',
 'weight': '250 lbs'}

A dictionary with information about LeBron James is returned. We can extract information even more precisely by specifying which field we are interested in. Let's get his salary:

In [54]:
all_players["los-angeles-lakers"]["LeBron James"]["salary"]

'$37,436,858'

In order to make data analysis easier, we can re-format this dictionary into a pandas DataFrame. The function pd.DataFrame.from_dict() can turn a dictionary of dictionaries into a pandas DataFrame:

In [88]:
import pandas as pd
gsw = pd.DataFrame.from_dict(all_players["golden-state-warriors"], orient = "index")
gsw

,uid,guid,id,height,weight,age,position,jersey,salary,birthDate,headshot,lastName,experience,college,birthPlace,hand
Alen Smailagic,s:40~l:46~a:4401415,6ed3f8924bfef2e70329ebd6a104ecae,4401415,"6' 10""",215 lbs,19,PF,6,"$898,310",08/18/00,https://a.espncdn.com/i/headshots/nba/players/...,Alen Smailagic,0,NaN,NaN,NaN
Andrew Wiggins,s:40~l:46~a:3059319,064c19d065276a21ca99fdfb296fe05d,3059319,"6' 7""",197 lbs,25,SF,22,"$27,504,630",02/23/95,https://a.espncdn.com/i/headshots/nba/players/...,Andrew Wiggins,5,Kansas,"Thornhill, ON",NaN
Chasson Randle,s:40~l:46~a:2580898,71b7154a3d81842448b623ee3e65d586,2580898,"6' 2""",185 lbs,27,PG,25,NaN,02/05/93,https://a.espncdn.com/i/headshots/nba/players/...,Chasson Randle,2,Stanford,"Rock Island, IL",NaN
Damion Lee,s:40~l:46~a:2595209,41fafb6d47a66d8f79f94161918541a4,2595209,"6' 5""",210 lbs,27,SG,1,"$842,327",10/21/92,https://a.espncdn.com/i/headshots/nba/players/...,Damion Lee,2,Louisville,NaN,L
Draymond Green,s:40~l:46~a:6589,de360720e41625f28a6bb5ff82616cb1,6589,"6' 6""",230 lbs,30,PF,23,"$18,539,130",03/04/90,https://a.espncdn.com/i/headshots/nba/players/...,Draymond Green,7,Michigan State,"Saginaw, MI",NaN
Eric Paschall,s:40~l:46~a:3133817,b67e5e0fa5cb209355845d165a49407e,3133817,"6' 6""",255 lbs,23,PF,7,"$898,310",11/04/96,https://a.espncdn.com/i/headshots/nba/players/...,Eric Paschall,0,Villanova,"North Tarrytown, NY",NaN
Jordan Poole,s:40~l:46~a:4277956,4b0492b5a52f267fe84098ef6d2e2bdf,4277956,"6' 4""",194 lbs,20,SG,3,"$1,964,760",06/19/99,https://a.espncdn.com/i/headshots/nba/players/...,Jordan Poole,0,Michigan,"Milwaukee, WI",B
Kevon Looney,s:40~l:46~a:3155535,10a8e77b877324c69966f0c4618caad6,3155535,"6' 9""",222 lbs,24,PF,5,"$4,464,226",02/06/96,https://a.espncdn.com/i/headshots/nba/players/...,Kevon Looney,4,UCLA,"Milwaukee, WI",NaN
Klay Thompson,s:40~l:46~a:6475,3411530a7ab7e8dce4f165d59a559520,6475,"6' 6""",215 lbs,30,SG,11,"$32,742,000",02/08/90,https://a.espncdn.com/i/headshots/nba/players/...,Klay Thompson,8,Washington State,"Los Angeles, CA",NaN
Ky Bowman,s:40~l:46~a:4065635,d0ef63e951bb5f842b7357521697dc62,4065635,"6' 1""",187 lbs,22,PG,12,"$350,189",06/17/97,https://a.espncdn.com/i/headshots/nba/players/...,Ky Bowman,0,Boston College,NaN,NaN


In the DataFrame above, each of the parameters such as 'age', 'salary', etc. are organized in columns and each player is a row. This makes display of the data much easier to read and understand. Furthermore, it also places null values when pieces of data are missing--for example, Chasson Randle's salary information is missing from the website, so 'NaN' is automatically placed in the DataFrame.

DataFrames allow us quickly make calculations, sort players based on their stats, and compare stats between teams. To make a DataFrame containing data from all the teams, we can loop through each team in `all_players`, construct DataFrames, label them with a `team` column, and aggregate them into a single DataFrame called `all_players_df`.

In [127]:
all_players_df = pd.DataFrame()

# loop through each team in all_players, create a pandas DataFrame, and append
for team in all_players.keys():
    team_df = pd.DataFrame.from_dict(all_players[team], orient = "index")
    team_df['team'] = team
    all_players_df = all_players_df.append(team_df)
    
all_players_df

,age,birthDate,birthPlace,college,experience,guid,hand,headshot,height,id,jersey,lastName,position,salary,team,uid,weight
Brad Wanamaker,30,07/25/89,"Philadelphia, PA",Pittsburgh,1,5aad35bbbb760e3958107639266768ae,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 3""",6507,9,Brad Wanamaker,PG,"$1,445,697",boston-celtics,s:40~l:46~a:6507,210 lbs
Carsen Edwards,22,03/12/98,"Houston, TX",Purdue,0,4b8ebdfd01221567925035c1e0d0c337,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"5' 11""",4066407,4,Carsen Edwards,PG,"$1,228,026",boston-celtics,s:40~l:46~a:4066407,200 lbs
Daniel Theis,27,04/04/92,Germany,NaN,2,ce75206c087f83ace6f9a8e3efbd9671,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 8""",2451037,27,Daniel Theis,C,"$5,000,000",boston-celtics,s:40~l:46~a:2451037,245 lbs
Enes Kanter,27,05/20/92,Switzerland,Kentucky,8,1e039b407b3daa6eeac69432aa6413fd,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 10""",6447,11,Enes Kanter,C,"$4,767,000",boston-celtics,s:40~l:46~a:6447,250 lbs
Gordon Hayward,30,03/23/90,"Indianapolis, IN",Butler,9,56f675cb8f40a5aaee5f5747ec9099c5,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 7""",4249,20,Gordon Hayward,SF,"$32,700,690",boston-celtics,s:40~l:46~a:4249,225 lbs
Grant Williams,21,11/30/98,"Houston, TX",Tennessee,0,3a93561dd9c3f1e8de40fbc7b40f7a5e,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 6""",4066218,12,Grant Williams,PF,"$2,379,840",boston-celtics,s:40~l:46~a:4066218,236 lbs
Javonte Green,26,07/23/93,"Alberta, VA",Radford,0,a4940ed033e0a114e8862f5a094aa3f8,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 4""",2596112,43,Javonte Green,SG,"$898,310",boston-celtics,s:40~l:46~a:2596112,205 lbs
Jaylen Brown,23,10/24/96,"Marietta, GA",California,3,0d5cde01f6d3225fdae544ef3304cda2,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 6""",3917376,7,Jaylen Brown,SG,"$6,534,829",boston-celtics,s:40~l:46~a:3917376,223 lbs
Jayson Tatum,22,03/03/98,NaN,Duke,2,ed3343b02ffaf6b4e4223a1920938c81,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 8""",4065648,0,Jayson Tatum,PF,"$7,830,000",boston-celtics,s:40~l:46~a:4065648,210 lbs
Kemba Walker,29,05/08/90,"Bronx, NY",Connecticut,8,665c55b2776846ac62a04efb4c9bcc80,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 0""",6479,8,Kemba Walker,PG,"$32,742,000",boston-celtics,s:40~l:46~a:6479,184 lbs


I'll export this data to a csv file, in case you readers out there want to read it in and play around with it yourself.

In [87]:
all_players_df.to_csv("NBA_roster_info_all_players_mar2020.csv")

## Scrape player stats: career averages

It would be nice to also have data coresponding to the performance of each player, in terms of points per game, field goal percentage, rebounds per game, etc. We can find this information at each player's personal page on ESPN:

![curry_stats_webpage.png](images/curry_stats_webpage.png)

We'll want to extract the career stats in the bottom row, which can be found in the highlighted section of the source code below:

![curry_stats_source.png](images/curry_stats_source.png)

In order to extract this information for each player in our DataFrame, we can construct URLs to open using the `id` column. Fortunately, the URL is standardized and very easy to construct. For example, using the `id` value of 3975 for Stephen Curry, the URL to open would be: https://www.espn.com/nba/player/stats/_/id/3975. Below is an example of extracting the career stats using regexes:

In [233]:
url = "https://www.espn.com/nba/player/stats/_/id/3975"
f = urllib.request.urlopen(url)
sleep(0.3)
player_source = f.read().decode('utf-8')
# extract career stats using this regex
stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
career_info = re.findall(stats_regex, player_source)
career_info


['"699","693","34.3","8.1-17.1","47.6","3.6-8.2","43.5","3.7-4.0","90.6","0.7","3.8","4.5","6.6","0.2","1.7","2.5","3.1","23.5"']

We observe that some of the stats are more than just numerical, such as "FT", which stands for "Free Throws Made-Attempted Per Game". We should split this up into two categories, "Free Throws Made (FTM)" and "Free Throws Attempted (FTA)", and do the same for field goals and 3 pointers. To do so, we can split the string based on "-" and then un-nest the list. Last, we need to convert the strings to floating point values.

In [ ]:
from itertools import chain
career_info = career_info[0].replace("\"", "").split(",")
career_info = list(chain.from_iterable([i.split("-") for i in career_info]))
career_info = list(map(float,career_info))
career_info

Now we can loop through each player in `all_players_df`, open their stats webpage, extract the career stats, and store the stats in a separate data frame called `career_stats_df` using the code below:

In [238]:
career_stats_df = pd.DataFrame(columns = ["GP","GS","MIN","FGM", "FGA","FG%","3PTM","3PTA","3P%","FTM","FTA","FT%","OR","DR","REB","AST","BLK","STL","PF","TO","PTS"])
for player_index in all_players_df.index:
    url = "https://www.espn.com/nba/player/stats/_/id/" + all_players_df.loc[player_index]['id']
    f = urllib.request.urlopen(url)
    sleep(0.3)
    player_source = f.read().decode('utf-8')
    # extract career stats using this regex
    stats_regex = ('\[\"Career\",\"\",(.*?)\]\},\{\"ttl\"\:\"Regular Season Totals\"')
    career_info = re.findall(stats_regex, player_source)
    try:
        # convert the stats to a list of floats, and add the entry to the DataFrame
        career_info = career_info[0].replace("\"", "").split(",")
        career_info = list(chain.from_iterable([i.split("-") for i in career_info]))
        career_info = list(map(float,career_info))
        career_stats_df = career_stats_df.append(pd.Series(career_info, index = career_stats_df.columns, name=player_index))
    except:
        # if no career stats were returned, the player was a rookie.
        print(player_index + " has no info, ", end = "") 

Brad Wanamaker, Carsen Edwards, Daniel Theis, Enes Kanter, Gordon Hayward, Grant Williams, Javonte Green, Jaylen Brown, Jayson Tatum, Kemba Walker, Marcus Smart, Romeo Langford, Semi Ojeleye, Tacko Fall, Tremont Waters, Vincent Poirier, Caris LeVert, Chris Chiozza, DeAndre Jordan, Dzanan Musa, Garrett Temple, Jarrett Allen, Jeremiah Martin, Joe Harris, Justin Anderson, Kevin Durant, Kyrie Irving, Nicolas Claxton, Rodions Kurucs, Spencer Dinwiddie, Taurean Prince, Theo Pinson, Wilson Chandler, Allonzo Trier, Bobby Portis, Damyean Dotson, Elfrid Payton, Frank Ntilikina, Ignas Brazdeikis, Julius Randle, Kadeem Allen, Kenny Wooten has no info, Maurice Harkless, Mitchell Robinson, RJ Barrett, Reggie Bullock, Taj Gibson, Wayne Ellington, Al Horford, Alec Burks, Ben Simmons, Furkan Korkmaz, Joel Embiid, Josh Richardson, Marial Shayok, Matisse Thybulle, Mike Scott, Norvel Pelle, Raul Neto, Shake Milton, Tobias Harris, Zhaire Smith, Chris Boucher, Dewan Hernandez, Fred VanVleet, Kyle Lowry, Mal

Some player webpages did not have career stats, which I found corresponded to rookies which have no games played. This threw an error in the loop, so I used a try/except clause to bypass the error and continue stripping content for the remaining players. The stats are currently stored in the object `career_stats_df`:

In [239]:
career_stats_df

,GP,GS,MIN,FGM,FGA,FG%,3PTM,3PTA,3P%,FTM,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
Brad Wanamaker,99.0,1.0,15.7,1.9,4.3,44.0,0.5,1.3,38.0,1.3,...,91.7,0.2,1.4,1.7,2.2,0.1,0.6,1.6,0.9,5.6
Carsen Edwards,35.0,0.0,9.0,1.1,3.2,32.7,0.6,1.9,30.9,0.3,...,84.6,0.2,1.1,1.2,0.6,0.1,0.3,0.9,0.4,3.0
Daniel Theis,187.0,62.0,17.2,2.6,4.7,55.4,0.4,1.1,34.0,1.1,...,75.3,1.6,3.1,4.7,1.2,0.9,0.5,2.8,0.7,6.7
Enes Kanter,634.0,222.0,21.8,4.8,8.8,54.2,0.1,0.2,28.7,2.0,...,77.6,2.9,4.7,7.6,0.9,0.5,0.4,2.2,1.5,11.6
Gordon Hayward,634.0,472.0,30.8,5.2,11.6,45.1,1.3,3.6,36.6,3.5,...,82.2,0.7,3.6,4.4,3.5,0.4,1.0,1.7,2.0,15.3
Grant Williams,62.0,5.0,15.6,1.3,3.1,41.7,0.3,1.4,24.7,0.6,...,72.5,0.9,1.7,2.7,1.0,0.5,0.5,2.4,0.7,3.5
Javonte Green,44.0,1.0,9.4,1.1,2.2,50.0,0.1,0.5,26.1,0.5,...,63.9,0.4,1.4,1.8,0.5,0.2,0.5,0.8,0.4,2.9
Jaylen Brown,272.0,165.0,26.1,4.8,10.3,47.0,1.4,3.7,37.0,1.9,...,68.0,0.8,3.6,4.4,1.4,0.3,0.8,2.4,1.5,12.9
Jayson Tatum,218.0,218.0,31.8,6.2,13.7,45.6,1.8,4.5,39.9,2.9,...,82.7,0.8,5.1,5.9,2.2,0.8,1.1,2.1,1.7,17.2
Kemba Walker,655.0,610.0,33.9,6.9,16.5,41.8,2.2,6.1,36.0,4.0,...,83.8,0.6,3.3,3.8,5.4,0.4,1.3,1.5,2.2,20.0


In [240]:
career_stats_df.to_csv("NBA_player_stats_all_mar2020.csv")

We will now join `career_stats_df` with `all_players_df`, which will merge the content from both data frames based on rows that have the same index (player name). Players in all_players_df that are not included in `career_stats_df` will have NaN values.

In [2]:
all_stats_df = all_players_df.join(career_stats_df)
all_stats_df

NameError: name 'all_players_df' is not defined

In [243]:
all_stats_df.to_csv("NBA_player_info_and_stats_joined_mar2020.csv")

Now that we have gathered and organized the roster data from each team, we can use the data to calculate some statistics. Let's start by calculating average stats per team, using `groupby()` with `mean()` in `pandas`.

In [8]:
all_stats_df = pd.read_csv("NBA_player_info_and_stats_joined_mar2020.csv", index_col=0)
all_stats_df

,age,birthDate,birthPlace,college,experience,guid,hand,headshot,height,id,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
Brad Wanamaker,30,07/25/89,"Philadelphia, PA",Pittsburgh,1,5aad35bbbb760e3958107639266768ae,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 3""",6507,...,91.7,0.2,1.4,1.7,2.2,0.1,0.6,1.6,0.9,5.6
Carsen Edwards,22,03/12/98,"Houston, TX",Purdue,0,4b8ebdfd01221567925035c1e0d0c337,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"5' 11""",4066407,...,84.6,0.2,1.1,1.2,0.6,0.1,0.3,0.9,0.4,3.0
Daniel Theis,27,04/04/92,Germany,NaN,2,ce75206c087f83ace6f9a8e3efbd9671,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 8""",2451037,...,75.3,1.6,3.1,4.7,1.2,0.9,0.5,2.8,0.7,6.7
Enes Kanter,27,05/20/92,Switzerland,Kentucky,8,1e039b407b3daa6eeac69432aa6413fd,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 10""",6447,...,77.6,2.9,4.7,7.6,0.9,0.5,0.4,2.2,1.5,11.6
Gordon Hayward,30,03/23/90,"Indianapolis, IN",Butler,9,56f675cb8f40a5aaee5f5747ec9099c5,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 7""",4249,...,82.2,0.7,3.6,4.4,3.5,0.4,1.0,1.7,2.0,15.3
Grant Williams,21,11/30/98,"Houston, TX",Tennessee,0,3a93561dd9c3f1e8de40fbc7b40f7a5e,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 6""",4066218,...,72.5,0.9,1.7,2.7,1.0,0.5,0.5,2.4,0.7,3.5
Javonte Green,26,07/23/93,"Alberta, VA",Radford,0,a4940ed033e0a114e8862f5a094aa3f8,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 4""",2596112,...,63.9,0.4,1.4,1.8,0.5,0.2,0.5,0.8,0.4,2.9
Jaylen Brown,23,10/24/96,"Marietta, GA",California,3,0d5cde01f6d3225fdae544ef3304cda2,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 6""",3917376,...,68.0,0.8,3.6,4.4,1.4,0.3,0.8,2.4,1.5,12.9
Jayson Tatum,22,03/03/98,NaN,Duke,2,ed3343b02ffaf6b4e4223a1920938c81,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 8""",4065648,...,82.7,0.8,5.1,5.9,2.2,0.8,1.1,2.1,1.7,17.2
Kemba Walker,29,05/08/90,"Bronx, NY",Connecticut,8,665c55b2776846ac62a04efb4c9bcc80,NaN,https://a.espncdn.com/i/headshots/nba/players/...,"6' 0""",6479,...,83.8,0.6,3.3,3.8,5.4,0.4,1.3,1.5,2.2,20.0


In [17]:
mean_df = all_stats_df.groupby('team').mean()
mean_df

,age,experience,id,jersey,GP,GS,MIN,FGM,FGA,FG%,...,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
team,,,,,,,,,,,,,,,,,,,,,
atlanta-hawks,25.538462,4.307692,3.020583e+06,12.692308,314.153846,200.076923,22.215385,3.815385,8.146154,48.253846,...,74.423077,1.223077,3.184615,4.407692,2.323077,0.569231,0.669231,2.146154,1.430769,10.300000
boston-celtics,25.000000,2.500000,2.753188e+06,30.375000,218.000000,121.687500,17.718750,2.787500,6.212500,45.575000,...,76.050000,0.712500,2.356250,3.081250,1.562500,0.356250,0.612500,1.631250,1.006250,7.662500
brooklyn-nets,25.882353,4.529412,2.423624e+06,12.529412,310.823529,214.000000,20.905882,3.482353,7.652941,44.123529,...,71.711765,0.864706,2.970588,3.835294,2.047059,0.476471,0.641176,1.688235,1.288235,9.582353
charlotte-hornets,24.266667,2.733333,3.090551e+06,13.666667,203.400000,103.466667,19.566667,2.626667,5.973333,43.206667,...,75.086667,0.933333,2.773333,3.680000,1.506667,0.440000,0.626667,1.746667,1.000000,7.013333
chicago-bulls,24.533333,2.533333,3.035302e+06,19.866667,199.400000,105.666667,20.033333,3.100000,6.866667,47.793333,...,73.613333,0.793333,2.580000,3.360000,1.826667,0.326667,0.726667,1.773333,1.020000,8.180000
cleveland-cavaliers,24.866667,2.933333,2.823630e+06,14.266667,253.615385,165.538462,20.376923,3.184615,6.953846,45.961538,...,62.484615,1.246154,3.069231,4.300000,1.638462,0.353846,0.546154,1.692308,1.200000,8.407692
dallas-mavericks,26.416667,3.250000,2.665610e+06,29.333333,237.750000,105.666667,19.083333,3.233333,7.058333,42.741667,...,70.833333,0.791667,2.725000,3.533333,1.808333,0.466667,0.575000,1.566667,1.033333,8.933333
denver-nuggets,25.928571,4.285714,2.803159e+06,16.642857,347.153846,185.307692,20.976923,3.423077,7.330769,45.884615,...,75.776923,1.061538,2.946154,3.984615,2.030769,0.469231,0.700000,1.846154,1.107692,9.092308
detroit-pistons,24.941176,3.411765,2.735654e+06,22.411765,235.058824,132.294118,18.876471,2.911765,6.652941,40.964706,...,64.917647,0.711765,2.335294,3.029412,1.723529,0.341176,0.517647,1.635294,1.000000,7.900000


As you can see, the index of the data frame that is returned corresponds to each individual team now, and the mean values are displayed for each of the columns with numerical values. To find the team with the highest averages for a specific stat, we can use the `sort_by()` function:

In [29]:
test = all_stats_df['salary'].values
[w.replace('\$|,', '') for w in test]

[str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 float,
 str,
 float,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 float,
 str,
 str,
 float,
 str,
 str,
 str,
 str,
 str,
 str,
 float,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,


Again, this is achieved by looping through all the team rosters on the ESPN website, then looping through all the players and extracting their salaries.

In general, webpages that link to subpages within the same site construct their links in some sort of standardized pattern. The techniques that I've outlined here should be broadly applicable for other websites. I hope what you've learned from this project will help you out on your own web scraping quests.

Thanks for reading! I hope my code helped you understand how to perform basic web scraping using Python.